In [1]:
import sys, os
from joblib import Parallel, delayed
import numpy as np
import matplotlib.pyplot as plt
import math
import scipy
from scipy.integrate import quad, nquad
from sklearn.preprocessing import normalize
from sktree.tree import DecisionTreeClassifier
from sktree.ensemble import HonestForestClassifier
from sktree.stats import FeatureImportanceForestClassifier
from sklearn.metrics import roc_auc_score
from scipy.stats import entropy

In [2]:
n_jobs = -1
test_size = 0.2
n_estimators = 500

### Calculate one dimention posterior with max_features = all, depth = 1
clf_Pertree = FeatureImportanceForestClassifier(
    estimator=HonestForestClassifier(
        n_estimators=n_estimators,
        max_features=0.3,
        bootstrap= False,
        tree_estimator=DecisionTreeClassifier(),
        # max_depth = 1,
        honest_fraction=0.5,
        n_jobs=n_jobs,

    ),
    test_size=test_size,
    # permute_per_tree=True,
    # sample_dataset_per_tree=True,
    stratify = True
)



In [22]:
def statistcs_Reps_Pertree_Linear(clf,n=100,p=4096,ratio=0.5,metric = 'mi',reps = 1):
    clf.reset()
    # coeffs = np.array([np.exp(-0.0022 * (i + 30)) if i < 10 else 0 for i in range(p)])
    coeffs = np.array([1/(i+5) if i <10 else 0 for i in range(p)])
    coeffs_noise = np.array([1 if i >=10 else 0 for i in range(p)])
    # print(coeffs)

    x_1 = np.random.normal(size=(n, p))
    noise = np.random.normal(size=(n, p))

    x_2 = x_1 * coeffs + noise*coeffs_noise
    x = np.nan_to_num(np.float32(np.vstack((x_1,x_2))))
    y = np.array([0]*n+[1]*n).reshape(-1,1)

    if metric == 'auc':
        stats,pos,samples = clf.statistic(x, y, metric=metric, return_posteriors=True,max_fpr = 0.1)
    else:
        stats,pos,samples = clf.statistic(x, y, metric=metric,return_posteriors=True)
    clf.reset()
    POS = pos[:,:,0].reshape((n_estimators,2*n))

    np.savetxt("/Users/baiyuxin/Desktop/JHU/NDD/Cancer/vs_samplesize/Posterior_linear_new_samplesize_p4096_{}_{}_{}.csv".format(metric,n, reps), POS, delimiter=",")
    np.savetxt("/Users/baiyuxin/Desktop/JHU/NDD/Cancer/vs_samplesize/Samples_linear_new_samplesize_p4096_{}_{}_{}.csv".format(metric,n, reps), samples, delimiter=",")
    #POSs.append(POS)
    clf.reset()
    return stats

In [3]:
sample_sizes = list(range(4,11,1))
# Square each element in the sequence
squared_values = [2 ** x for x in sample_sizes]
print(len(squared_values))
REPs = 10

7


In [6]:
statistcs_Reps_Pertree_Linear(clf_Pertree,1024,p=2048,ratio=0.5,metric = 'mi',reps = 1)

0.5460184603496929

In [24]:
Stats_Paucs_Samplesize_Linear = np.zeros((REPs,len(squared_values)))

for i in range(0,REPs):
    stats_paucs_samplesize_linear =[]
    for n in squared_values:
        stat = statistcs_Reps_Pertree_Linear(clf_Pertree,n,p=4096,ratio=0.5,metric = 'auc',reps = i)
        # print(stat)
        stats_paucs_samplesize_linear.append(stat)
    print(stats_paucs_samplesize_linear)
    np.savetxt("/Users/baiyuxin/Desktop/JHU/NDD/Cancer/vs_samplesize/linear_new_pAUC_samplesize_p4096_{}.csv".format(i), stats_paucs_samplesize_linear, delimiter=",")
    Stats_Paucs_Samplesize_Linear[i,:] = stats_paucs_samplesize_linear

[0.48601973684210525, 0.717310855263158, 1.0, 1.0, 0.9999196905838816, 1.0, 1.0]
[0.506578947368421, 0.5877878289473685, 1.0, 0.9996787623355263, 1.0, 1.0, 1.0]
[0.5394736842105263, 0.5703125, 1.0, 1.0, 1.0, 1.0, 0.9999849419844777]
[0.5106907894736842, 0.6618009868421053, 1.0, 1.0, 1.0, 0.9999799226459705, 1.0]
[0.5394736842105263, 0.5497532894736842, 0.994860197368421, 1.0, 1.0, 1.0, 1.0]
[0.5189144736842105, 0.6083470394736842, 1.0, 1.0, 1.0, 0.9999598452919407, 1.0]
[0.5106907894736842, 0.5826480263157895, 1.0, 1.0, 1.0, 1.0, 1.0]
[0.47368421052631576, 0.5281661184210527, 1.0, 1.0, 1.0, 0.9999799226459705, 1.0]
[0.5189144736842105, 0.5384457236842105, 0.9917763157894737, 1.0, 1.0, 0.9999799226459705, 0.9999949806614926]
[0.506578947368421, 0.6648848684210527, 1.0, 1.0, 1.0, 1.0, 1.0]


In [25]:
Stats_Paucs_Linear = np.zeros((REPs,len(squared_values)))
for i in range(REPs):
    pauc = np.array(np.genfromtxt('/Users/baiyuxin/Desktop/JHU/NDD/Cancer/vs_samplesize/linear_new_pAUC_samplesize_p4096_{}.csv'.format(i),delimiter=','))
    Stats_Paucs_Linear[i,:] = pauc
print(Stats_Paucs_Linear)


Stats_MI_Linear = np.zeros((REPs,len(squared_values)))
Stats_Conen_Linear = np.zeros((REPs,len(squared_values)))
for i in range(REPs):
    for samp in range(len(squared_values)):
        pos = np.genfromtxt('/Users/baiyuxin/Desktop/JHU/NDD/Cancer/vs_samplesize/Posterior_linear_new_samplesize_p4096_auc_{}_{}.csv'.format(squared_values[samp],i),delimiter=',')
        posterior_forest_0 = np.nanmean(pos, axis=0)
        posterior_forest_1 = np.ones(posterior_forest_0.shape)-posterior_forest_0
        posterior_forest = np.hstack((posterior_forest_0.reshape(-1,1),posterior_forest_1.reshape(-1,1)))

        stats_conen = np.mean(entropy(posterior_forest, base=np.exp(1), axis=1))

        H_Y = entropy([50,50], base=np.exp(1))
        stats_mi = H_Y - stats_conen

        Stats_Conen_Linear[i,samp] = stats_conen
        Stats_MI_Linear[i,samp] = stats_mi

[[0.48601974 0.71731086 1.         1.         0.99991969 1.
  1.        ]
 [0.50657895 0.58778783 1.         0.99967876 1.         1.
  1.        ]
 [0.53947368 0.5703125  1.         1.         1.         1.
  0.99998494]
 [0.51069079 0.66180099 1.         1.         1.         0.99997992
  1.        ]
 [0.53947368 0.54975329 0.9948602  1.         1.         1.
  1.        ]
 [0.51891447 0.60834704 1.         1.         1.         0.99995985
  1.        ]
 [0.51069079 0.58264803 1.         1.         1.         1.
  1.        ]
 [0.47368421 0.52816612 1.         1.         1.         0.99997992
  1.        ]
 [0.51891447 0.53844572 0.99177632 1.         1.         0.99997992
  0.99999498]
 [0.50657895 0.66488487 1.         1.         1.         1.
  1.        ]]


In [26]:
np.savetxt("/Users/baiyuxin/Desktop/JHU/NDD/Cancer/vs_samplesize/Stats_Paucs_SampleSize_Linear_new_p4096_MIGHT.csv", Stats_Paucs_Linear, delimiter=",")
np.savetxt("/Users/baiyuxin/Desktop/JHU/NDD/Cancer/vs_samplesize/Stats_Conen_SampleSize_Linear_new_p4096_MIGHT.csv", Stats_Conen_Linear, delimiter=",")
np.savetxt("/Users/baiyuxin/Desktop/JHU/NDD/Cancer/vs_samplesize/Stats_MI_SampleSize_Linear_new_p4096_MIGHT.csv", Stats_MI_Linear, delimiter=",")
Stats_MI_Linear

array([[8.53834431e-04, 5.01743866e-03, 4.45553822e-02, 2.81024085e-01,
        4.20854241e-01, 4.77592681e-01, 5.29208599e-01],
       [8.32450348e-04, 7.89038303e-04, 5.74077773e-02, 2.61395613e-01,
        3.90445911e-01, 4.73328825e-01, 5.39676953e-01],
       [6.11777910e-04, 2.59380608e-03, 8.59656375e-02, 2.76963125e-01,
        3.92850062e-01, 4.91393034e-01, 5.32973570e-01],
       [4.95817404e-04, 2.91788944e-03, 4.77196316e-02, 2.59339128e-01,
        4.01773784e-01, 4.79513106e-01, 5.45196228e-01],
       [7.48012298e-04, 1.08676777e-03, 5.13788147e-02, 2.62585309e-01,
        4.03989461e-01, 4.92916451e-01, 5.29093490e-01],
       [1.05442721e-03, 1.69819058e-03, 9.54529718e-02, 2.68406923e-01,
        3.90294870e-01, 4.86946598e-01, 5.40716466e-01],
       [7.58220248e-04, 1.04590984e-03, 5.94072471e-02, 2.79129967e-01,
        3.90205854e-01, 4.82358207e-01, 5.38442590e-01],
       [7.69843100e-04, 1.44824056e-03, 9.00884545e-02, 2.74578209e-01,
        4.08511666e-01, 4

In [8]:
##Add Logistics Regression for linear
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
REPs = 10
p = 2048

Stats_MI_LG = np.zeros((REPs,len(squared_values)))
Stats_Conen_LG = np.zeros((REPs,len(squared_values)))
Stats_Pauc_LG = np.zeros((REPs,len(squared_values)))


for i in range(0,REPs):
    print(i)
    stats_paucs_samplesize_mu0 =[]
    stats_mi_samplesize_mu0 =[]
    stats_conen_samplesize_mu0 =[]
    for n in squared_values:
        coeffs = np.array([np.exp(-0.0022 * (i + 30)) if i < 10 else 0 for i in range(p)])
        coeffs_noise = np.array([0 if i <10 else 1 for i in range(p)])

        x_1 = np.random.normal(size=(n, p))
        noise = np.random.normal(size=(n, p))
        x_2 = x_1* coeffs + noise*coeffs_noise

        # x_2 = x_1 * coeffs + noise*coeffs_noise
        x = np.vstack((x_1,x_2))
        y = np.array([0]*n+[1]*n).reshape(-1,1).ravel()
        
        clf = LogisticRegression(penalty='l1',solver = 'liblinear').fit(x, y)
        # print(clf.score(x, y))
        posterior = clf.predict_proba(x)

        stats_conen = np.mean(entropy(posterior, base=np.exp(1), axis=1))
        # print(stats_conen)

        # _, counts = np.unique(y_true_final, return_counts=True)
        H_Y = entropy([50,50], base=np.exp(1))
        # print(H_Y)
        stats_mi = H_Y - stats_conen

        pauc = roc_auc_score(
                y, posterior[:,1], max_fpr=0.1
            )
        print(stats_conen,stats_mi,pauc)
        
        stats_paucs_samplesize_mu0.append(pauc)
        stats_mi_samplesize_mu0.append(stats_mi)
        stats_conen_samplesize_mu0.append(stats_conen)

    Stats_Pauc_LG[i,:] = stats_paucs_samplesize_mu0
    Stats_MI_LG[i,:] = stats_mi_samplesize_mu0
    Stats_Conen_LG[i,:] = stats_conen_samplesize_mu0
    

0
0.2256214071950056 0.4675257733649397 1.0
0.18266556487157387 0.5104816156883714 1.0
0.15019448061332197 0.5429526999466233 1.0
0.12617920399254745 0.5669679765673978 1.0
0.10721188850175528 0.58593529205819 1.0
0.09509395076686862 0.5980532297930766 1.0
1
0.22358597969154703 0.46956120086839825 1.0
0.18550611901329547 0.5076410615466498 1.0
0.15149792847996804 0.5416492520799773 1.0
0.12364156985572787 0.5695056107042175 1.0
0.10847429376096421 0.5846728867989811 1.0
0.09131863508145008 0.6018285454784952 1.0
2
0.23005395504234546 0.4630932255175998 1.0
0.17490756894376175 0.5182396116161836 1.0
0.15333410489609284 0.5398130756638524 1.0
0.12116205130389006 0.5719851292560553 1.0
0.10782142569361058 0.5853257548663346 1.0
0.09071080842814477 0.6024363721318005 1.0
3
0.22573320501505154 0.4674139755448937 1.0
0.189025947409839 0.5041212331501063 1.0
0.15050326971027056 0.5426439108496748 1.0
0.12529580698398843 0.5678513735759568 1.0
0.10572319343261516 0.5874239871273301 1.0
0.09377

In [11]:
np.savetxt("/Users/baiyuxin/Desktop/JHU/NDD/Cancer/vs_samplesize/Stats_Paucs_SampleSize_Linear_p2048_LG.csv", Stats_Pauc_LG, delimiter=",")
np.savetxt("/Users/baiyuxin/Desktop/JHU/NDD/Cancer/vs_samplesize/Stats_Conen_SampleSize_Linear_p2048_LG.csv", Stats_Conen_LG, delimiter=",")
np.savetxt("/Users/baiyuxin/Desktop/JHU/NDD/Cancer/vs_samplesize/Stats_MI_SampleSize_Linear_p2048_LG.csv", Stats_MI_LG, delimiter=",")
Stats_MI_LG

array([[0.68117918, 0.6813614 , 0.68075512, 0.68079606, 0.68037258,
        0.67955816],
       [0.68148812, 0.68144839, 0.68088894, 0.6807253 , 0.68062093,
        0.67938142],
       [0.68126716, 0.68103911, 0.68147376, 0.68119524, 0.68022494,
        0.67880468],
       [0.68131856, 0.68123968, 0.68115255, 0.68088773, 0.68048998,
        0.67908884],
       [0.6810976 , 0.68160049, 0.68113251, 0.68069813, 0.68050426,
        0.67910916],
       [0.68111745, 0.68133427, 0.6812511 , 0.68085048, 0.68048165,
        0.67962793],
       [0.68127526, 0.68119104, 0.68121414, 0.68101918, 0.68035177,
        0.67964376],
       [0.68129505, 0.68120841, 0.68143598, 0.68094677, 0.68049065,
        0.67917327],
       [0.68109696, 0.68119105, 0.68106536, 0.68063566, 0.68052266,
        0.67877488],
       [0.68134337, 0.68134066, 0.68123822, 0.68092045, 0.6800935 ,
        0.6790455 ],
       [0.68107789, 0.68154806, 0.68121122, 0.68060759, 0.68021537,
        0.67895124],
       [0.68144493, 0

In [12]:
### Add SVM to the Linear
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import cross_val_score, StratifiedKFold, cross_val_predict

REPs = 10
p = 2048

Stats_MI_SVM = np.zeros((REPs,len(squared_values)))
Stats_Conen_SVM = np.zeros((REPs,len(squared_values)))
Stats_Pauc_SVM = np.zeros((REPs,len(squared_values)))


for i in range(0,REPs):
    print(i)
    stats_paucs_samplesize_mu0 =[]
    stats_mi_samplesize_mu0 =[]
    stats_conen_samplesize_mu0 =[]
    for n in squared_values:
        
        coeffs = np.array([np.exp(-0.0022 * (i + 30)) if i < 10 else 0 for i in range(p)])
        coeffs_noise = np.array([1 if i >=10 else 0 for i in range(p)])

        x_1 = np.random.normal(size=(n, p))
        noise = np.random.normal(size=(n, p))

        x_2 = x_1 * coeffs + noise*coeffs_noise

        x = np.vstack((x_1,x_2))
        y = np.array([0]*n+[1]*n).reshape(-1,1).ravel()
        
        model = SVC(probability=True,kernel = 'rbf')  # Set probability=True to enable probability estimates
        model.fit(x, y)
        posterior = model.predict_proba(x)

        stats_conen = np.mean(entropy(posterior, base=np.exp(1), axis=1))
        # print(stats_conen)

        # _, counts = np.unique(y_true_final, return_counts=True)
        H_Y = entropy([50,50], base=np.exp(1))
        # print(H_Y)
        stats_mi = H_Y - stats_conen

        pauc = roc_auc_score(
                y, posterior[:,1], max_fpr=0.1
            )
        print(stats_conen,stats_mi,pauc)
        
        stats_paucs_samplesize_mu0.append(pauc)
        stats_mi_samplesize_mu0.append(stats_mi)
        stats_conen_samplesize_mu0.append(stats_conen)

    Stats_Pauc_SVM[i,:] = stats_paucs_samplesize_mu0
    Stats_MI_SVM[i,:] = stats_mi_samplesize_mu0
    Stats_Conen_SVM[i,:] = stats_conen_samplesize_mu0
    

0
0.41664658964876644 0.27650059091117885 0.47368421052631576
0.3330308351871497 0.3601163453727956 0.47368421052631576
0.39218705293002565 0.30096012762991964 0.47368421052631576
0.5082378684540024 0.18490931210594286 0.47368421052631576
0.6637871684712782 0.02936001208866712 0.47368421052631576
0.6929493363359794 0.00019784422396584844 0.47368421052631576
1
0.3128749911369034 0.38027218942304186 0.47368421052631576
0.5593164576944203 0.133830722865525 0.47368421052631576
0.48325001047926175 0.20989717008068354 0.47368421052631576
0.5735761639634936 0.11957101659645164 0.47368421052631576
0.6203279528423007 0.0728192277176446 0.47368421052631576
0.585123636643357 0.1080235439165883 0.47368421052631576
2
0.31335139343314733 0.37979578712679796 0.47368421052631576
0.45588934636688605 0.23725783419305924 0.47368421052631576
0.4828108817165768 0.21033629884336846 0.47368421052631576
0.5205852565708378 0.17256192398910752 0.47368421052631576
0.5678120267310915 0.12533515382885374 0.4736842

In [13]:
np.savetxt("/Users/baiyuxin/Desktop/JHU/NDD/Cancer/vs_samplesize/Stats_Paucs_SampleSize_Linear_p2048_SVM.csv", Stats_Pauc_SVM, delimiter=",")
np.savetxt("/Users/baiyuxin/Desktop/JHU/NDD/Cancer/vs_samplesize/Stats_Conen_SampleSize_Linear_p2048_SVM.csv", Stats_Conen_SVM, delimiter=",")
np.savetxt("/Users/baiyuxin/Desktop/JHU/NDD/Cancer/vs_samplesize/Stats_MI_SampleSize_Linear_p2048_SVM.csv", Stats_MI_SVM, delimiter=",")
Stats_MI_SVM

array([[2.76500591e-01, 3.60116345e-01, 3.00960128e-01, 1.84909312e-01,
        2.93600121e-02, 1.97844224e-04],
       [3.80272189e-01, 1.33830723e-01, 2.09897170e-01, 1.19571017e-01,
        7.28192277e-02, 1.08023544e-01],
       [3.79795787e-01, 2.37257834e-01, 2.10336299e-01, 1.72561924e-01,
        1.25335154e-01, 3.51890653e-02],
       [4.09635318e-01, 3.48310905e-01, 1.02417388e-01, 1.11268822e-01,
        6.16892778e-02, 3.50540156e-04],
       [3.44133693e-01, 3.10806123e-01, 2.63693259e-01, 1.15900058e-01,
        1.66036830e-01, 8.05000008e-02],
       [4.26641664e-01, 3.26931786e-01, 2.99187122e-01, 1.76863053e-03,
        9.57326245e-02, 6.85109749e-03],
       [3.88976457e-01, 4.10053445e-01, 1.18089952e-01, 2.39813125e-02,
        1.13928719e-01, 4.66350607e-02],
       [3.73798801e-01, 2.64731227e-01, 1.93430734e-01, 2.15886372e-01,
        3.29936487e-02, 9.93396327e-04],
       [3.98125116e-01, 6.08689823e-02, 2.72363292e-01, 8.93308722e-02,
        1.67722641e-01, 

In [28]:
#!/usr/bin/env python
# Written by Greg Ver Steeg
# See readme.pdf for documentation
# Or go to http://www.isi.edu/~gregv/npeet.html

import warnings

import numpy as np
import numpy.linalg as la
from numpy import log
from scipy.special import digamma
from sklearn.neighbors import BallTree, KDTree

# CONTINUOUS ESTIMATORS


def entropy(x, k=3, base=2):
    """The classic K-L k-nearest neighbor continuous entropy estimator
    x should be a list of vectors, e.g. x = [[1.3], [3.7], [5.1], [2.4]]
    if x is a one-dimensional scalar and we have four samples
    """
    assert k <= len(x) - 1, "Set k smaller than num. samples - 1"
    x = np.asarray(x)
    n_elements, n_features = x.shape
    x = add_noise(x)
    tree = build_tree(x)
    nn = query_neighbors(tree, x, k)
    const = digamma(n_elements) - digamma(k) + n_features * log(2)
    return (const + n_features * np.log(nn).mean()) / log(base)


def centropy(x, y, k=3, base=2):
    """The classic K-L k-nearest neighbor continuous entropy estimator for the
    entropy of X conditioned on Y.
    """
    xy = np.c_[x, y]
    entropy_union_xy = entropy(xy, k=k, base=base)
    entropy_y = entropy(y, k=k, base=base)
    return entropy_union_xy - entropy_y


def tc(xs, k=3, base=2):
    xs_columns = np.expand_dims(xs, axis=0).T
    entropy_features = [entropy(col, k=k, base=base) for col in xs_columns]
    return np.sum(entropy_features) - entropy(xs, k, base)


def ctc(xs, y, k=3, base=2):
    xs_columns = np.expand_dims(xs, axis=0).T
    centropy_features = [centropy(col, y, k=k, base=base) for col in xs_columns]
    return np.sum(centropy_features) - centropy(xs, y, k, base)


def corex(xs, ys, k=3, base=2):
    xs_columns = np.expand_dims(xs, axis=0).T
    cmi_features = [mi(col, ys, k=k, base=base) for col in xs_columns]
    return np.sum(cmi_features) - mi(xs, ys, k=k, base=base)


def mi(x, y, z=None, k=3, base=2, alpha=0):
    """Mutual information of x and y (conditioned on z if z is not None)
    x, y should be a list of vectors, e.g. x = [[1.3], [3.7], [5.1], [2.4]]
    if x is a one-dimensional scalar and we have four samples
    """
    assert len(x) == len(y), "Arrays should have same length"
    assert k <= len(x) - 1, "Set k smaller than num. samples - 1"
    x, y = np.asarray(x), np.asarray(y)
    x, y = x.reshape(x.shape[0], -1), y.reshape(y.shape[0], -1)
    x = add_noise(x)
    y = add_noise(y)
    points = [x, y]
    if z is not None:
        z = np.asarray(z)
        z = z.reshape(z.shape[0], -1)
        points.append(z)
    points = np.hstack(points)
    # Find nearest neighbors in joint space, p=inf means max-norm
    tree = build_tree(points)
    dvec = query_neighbors(tree, points, k)
    if z is None:
        a, b, c, d = (
            avgdigamma(x, dvec),
            avgdigamma(y, dvec),
            digamma(k),
            digamma(len(x)),
        )
        if alpha > 0:
            d += lnc_correction(tree, points, k, alpha)
    else:
        xz = np.c_[x, z]
        yz = np.c_[y, z]
        a, b, c, d = (
            avgdigamma(xz, dvec),
            avgdigamma(yz, dvec),
            avgdigamma(z, dvec),
            digamma(k),
        )
    return (-a - b + c + d) / log(base)


def cmi(x, y, z, k=3, base=2):
    """Mutual information of x and y, conditioned on z
    Legacy function. Use mi(x, y, z) directly.
    """
    return mi(x, y, z=z, k=k, base=base)


def kldiv(x, xp, k=3, base=2):
    """KL Divergence between p and q for x~p(x), xp~q(x)
    x, xp should be a list of vectors, e.g. x = [[1.3], [3.7], [5.1], [2.4]]
    if x is a one-dimensional scalar and we have four samples
    """
    assert k < min(len(x), len(xp)), "Set k smaller than num. samples - 1"
    assert len(x[0]) == len(xp[0]), "Two distributions must have same dim."
    x, xp = np.asarray(x), np.asarray(xp)
    x, xp = x.reshape(x.shape[0], -1), xp.reshape(xp.shape[0], -1)
    d = len(x[0])
    n = len(x)
    m = len(xp)
    const = log(m) - log(n - 1)
    tree = build_tree(x)
    treep = build_tree(xp)
    nn = query_neighbors(tree, x, k)
    nnp = query_neighbors(treep, x, k - 1)
    return (const + d * (np.log(nnp).mean() - np.log(nn).mean())) / log(base)


def lnc_correction(tree, points, k, alpha):
    e = 0
    n_sample = points.shape[0]
    for point in points:
        # Find k-nearest neighbors in joint space, p=inf means max norm
        knn = tree.query(point[None, :], k=k + 1, return_distance=False)[0]
        knn_points = points[knn]
        # Substract mean of k-nearest neighbor points
        knn_points = knn_points - knn_points[0]
        # Calculate covariance matrix of k-nearest neighbor points, obtain eigen vectors
        covr = knn_points.T @ knn_points / k
        _, v = la.eig(covr)
        # Calculate PCA-bounding box using eigen vectors
        V_rect = np.log(np.abs(knn_points @ v).max(axis=0)).sum()
        # Calculate the volume of original box
        log_knn_dist = np.log(np.abs(knn_points).max(axis=0)).sum()

        # Perform local non-uniformity checking and update correction term
        if V_rect < log_knn_dist + np.log(alpha):
            e += (log_knn_dist - V_rect) / n_sample
    return e


# DISCRETE ESTIMATORS
def entropyd(sx, base=2):
    """Discrete entropy estimator
    sx is a list of samples
    """
    unique, count = np.unique(sx, return_counts=True, axis=0)
    # Convert to float as otherwise integer division results in all 0 for proba.
    proba = count.astype(float) / len(sx)
    # Avoid 0 division; remove probabilities == 0.0 (removing them does not change the entropy estimate as 0 * log(1/0) = 0.
    proba = proba[proba > 0.0]
    return np.sum(proba * np.log(1.0 / proba)) / log(base)


def midd(x, y, base=2):
    """Discrete mutual information estimator
    Given a list of samples which can be any hashable object
    """
    assert len(x) == len(y), "Arrays should have same length"
    return entropyd(x, base) - centropyd(x, y, base)


def cmidd(x, y, z, base=2):
    """Discrete mutual information estimator
    Given a list of samples which can be any hashable object
    """
    assert len(x) == len(y) == len(z), "Arrays should have same length"
    xz = np.c_[x, z]
    yz = np.c_[y, z]
    xyz = np.c_[x, y, z]
    return (
        entropyd(xz, base)
        + entropyd(yz, base)
        - entropyd(xyz, base)
        - entropyd(z, base)
    )


def centropyd(x, y, base=2):
    """The classic K-L k-nearest neighbor continuous entropy estimator for the
    entropy of X conditioned on Y.
    """
    xy = np.c_[x, y]
    return entropyd(xy, base) - entropyd(y, base)


def tcd(xs, base=2):
    xs_columns = np.expand_dims(xs, axis=0).T
    entropy_features = [entropyd(col, base=base) for col in xs_columns]
    return np.sum(entropy_features) - entropyd(xs, base)


def ctcd(xs, y, base=2):
    xs_columns = np.expand_dims(xs, axis=0).T
    centropy_features = [centropyd(col, y, base=base) for col in xs_columns]
    return np.sum(centropy_features) - centropyd(xs, y, base)


def corexd(xs, ys, base=2):
    xs_columns = np.expand_dims(xs, axis=0).T
    cmi_features = [midd(col, ys, base=base) for col in xs_columns]
    return np.sum(cmi_features) - midd(xs, ys, base)


# MIXED ESTIMATORS
def micd(x, y, k=3, base=2, warning=True):
    """If x is continuous and y is discrete, compute mutual information"""
    assert len(x) == len(y), "Arrays should have same length"
    entropy_x = entropy(x, k, base)

    y_unique, y_count = np.unique(y, return_counts=True, axis=0)
    y_proba = y_count / len(y)

    entropy_x_given_y = 0.0
    for yval, py in zip(y_unique, y_proba):
        x_given_y = x[(y == yval).all(axis=1)]
        if k <= len(x_given_y) - 1:
            entropy_x_given_y += py * entropy(x_given_y, k, base)
        else:
            if warning:
                warnings.warn(
                    "Warning, after conditioning, on y={yval} insufficient data. "
                    "Assuming maximal entropy in this case.".format(yval=yval)
                )
            entropy_x_given_y += py * entropy_x
    return abs(entropy_x - entropy_x_given_y)  # units already applied


def midc(x, y, k=3, base=2, warning=True):
    return micd(y, x, k, base, warning)


def centropycd(x, y, k=3, base=2, warning=True):
    return entropy(x, base) - micd(x, y, k, base, warning)


def centropydc(x, y, k=3, base=2, warning=True):
    return centropycd(y, x, k=k, base=base, warning=warning)


def ctcdc(xs, y, k=3, base=2, warning=True):
    xs_columns = np.expand_dims(xs, axis=0).T
    centropy_features = [
        centropydc(col, y, k=k, base=base, warning=warning) for col in xs_columns
    ]
    return np.sum(centropy_features) - centropydc(xs, y, k, base, warning)


def ctccd(xs, y, k=3, base=2, warning=True):
    return ctcdc(y, xs, k=k, base=base, warning=warning)


def corexcd(xs, ys, k=3, base=2, warning=True):
    return corexdc(ys, xs, k=k, base=base, warning=warning)


def corexdc(xs, ys, k=3, base=2, warning=True):
    return tcd(xs, base) - ctcdc(xs, ys, k, base, warning)


# UTILITY FUNCTIONS


def add_noise(x, intens=1e-10):
    # small noise to break degeneracy, see doc.
    return x + intens * np.random.random_sample(x.shape)


def query_neighbors(tree, x, k):
    return tree.query(x, k=k + 1)[0][:, k]


def count_neighbors(tree, x, r):
    return tree.query_radius(x, r, count_only=True)


def avgdigamma(points, dvec):
    # This part finds number of neighbors in some radius in the marginal space
    # returns expectation value of <psi(nx)>
    tree = build_tree(points)
    dvec = dvec - 1e-15
    num_points = count_neighbors(tree, points, dvec)
    return np.mean(digamma(num_points))


def build_tree(points):
    if points.shape[1] >= 20:
        return BallTree(points, metric="chebyshev")
    return KDTree(points, metric="chebyshev")


# TESTS


def shuffle_test(measure, x, y, z=False, ns=200, ci=0.95, **kwargs):
    """Shuffle test
    Repeatedly shuffle the x-values and then estimate measure(x, y, [z]).
    Returns the mean and conf. interval ('ci=0.95' default) over 'ns' runs.
    'measure' could me mi, cmi, e.g. Keyword arguments can be passed.
    Mutual information and CMI should have a mean near zero.
    """
    x_clone = np.copy(x)  # A copy that we can shuffle
    outputs = []
    for i in range(ns):
        np.random.shuffle(x_clone)
        if z:
            outputs.append(measure(x_clone, y, z, **kwargs))
        else:
            outputs.append(measure(x_clone, y, **kwargs))
    outputs.sort()
    return np.mean(outputs), (
        outputs[int((1.0 - ci) / 2 * ns)],
        outputs[int((1.0 + ci) / 2 * ns)],
    )


if __name__ == "__main__":
    print("MI between two independent continuous random variables X and Y:")
    np.random.seed(0)
    x = np.random.randn(1000, 10)
    y = np.random.randn(1000, 3)
    print(mi(x, y, base=2, alpha=0))

MI between two independent continuous random variables X and Y:
-0.022484075103376248


In [29]:
### KSG for Mutual Information
from sktree.experimental import mutual_info_ksg
from sklearn.feature_selection import mutual_info_classif
from sklearn.feature_selection import mutual_info_regression
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import cross_val_score, StratifiedKFold, cross_val_predict
from sktree.ensemble import UnsupervisedObliqueRandomForest
from sktree.tree import compute_forest_similarity_matrix


REPs = 10
p = 4096

Stats_MI_KSG = np.zeros((REPs,len(squared_values)))

for i in range(0,REPs):
    print(i)
    stats_mi_samplesize_mu0 =[]
    for n in squared_values:
        coeffs = np.array([1/(i+5) if i <10 else 0 for i in range(p)])
        coeffs_noise = np.array([1 if i >=10 else 0 for i in range(p)])

        x_1 = np.random.normal(size=(int(n), p))
        noise = np.random.normal(size=(int(n), p))

        x_2 = x_1 * coeffs + noise*coeffs_noise
        # print(x_2)

        x = np.vstack((x_1,x_2))
        y = np.array([0]*n+[1]*n).reshape(-1,1)

        # stats_mi = mutual_info_ksg(x_1,x_2,k = 0.2)
        # print(scipy.special.digamma(n)/2+stats_mi/2+scipy.special.digamma(n*0.2)/2)
        # stats_mi = mutual_info_ksg(x,y,metric='euclidean',algorithm = 'kd_tree',k = 0.5)
        stats_mi = mi(x,y,k = int(np.sqrt(n)))
        # stats_mi = mutual_info_classif(x,y,n_neighbors= int(np.sqrt(n)))
        # stats_mi = mutual_info_classif(x,y,n_neighbors= int(np.sqrt(n)))
        # print(stats_mi)
        # print(sum(stats_mi))
        # print(sum(stats_mi)/p)
        # print(np.sum(stats_mi)-(p-1)*np.mean(digamma(int(np.sqrt(n))))-digamma(2*n)+(p-1)*np.mean(digamma(n)))
        # print((np.sum(stats_mi)-(p-1)*np.mean(digamma(int(np.sqrt(n))))-digamma(2*n)+(p-1)*np.mean(digamma(n)))/p)
        stats_mi_samplesize_mu0.append(np.mean(stats_mi))
    print(stats_mi_samplesize_mu0)
    Stats_MI_KSG[i,:] = stats_mi_samplesize_mu0

0
[0.0, 0.0, 0.0, 2.562741203051934e-15, -2.562741203051934e-15, -2.562741203051934e-15, 2.562741203051934e-15]
1
[0.0, 0.0, 0.0, 2.562741203051934e-15, -2.562741203051934e-15, -2.562741203051934e-15, 2.562741203051934e-15]
2
[0.0, 0.0, 0.0, 2.562741203051934e-15, -2.562741203051934e-15, -2.562741203051934e-15, 2.562741203051934e-15]
3
[0.0, 0.0, 0.0, 2.562741203051934e-15, -2.562741203051934e-15, -2.562741203051934e-15, 2.562741203051934e-15]
4
[0.0, 0.0, 0.0, 2.562741203051934e-15, -2.562741203051934e-15, -2.562741203051934e-15, 2.562741203051934e-15]
5
[0.0, 0.0, 0.0, 2.562741203051934e-15, -2.562741203051934e-15, -2.562741203051934e-15, 2.562741203051934e-15]
6
[0.0, 0.0, 0.0, 2.562741203051934e-15, -2.562741203051934e-15, -2.562741203051934e-15, 2.562741203051934e-15]
7
[0.0, 0.0, 0.0, 2.562741203051934e-15, -2.562741203051934e-15, -2.562741203051934e-15, 2.562741203051934e-15]
8
[0.0, 0.0, 0.0, 2.562741203051934e-15, -2.562741203051934e-15, -2.562741203051934e-15, 2.5627412030519

In [30]:
np.savetxt("/Users/baiyuxin/Desktop/JHU/NDD/Cancer/vs_samplesize/Stats_MI_SampleSize_Linear_new_p4096_KSG.csv", Stats_MI_KSG, delimiter=",")
Stats_MI_KSG

array([[ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  2.5627412e-15,
        -2.5627412e-15, -2.5627412e-15,  2.5627412e-15],
       [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  2.5627412e-15,
        -2.5627412e-15, -2.5627412e-15,  2.5627412e-15],
       [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  2.5627412e-15,
        -2.5627412e-15, -2.5627412e-15,  2.5627412e-15],
       [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  2.5627412e-15,
        -2.5627412e-15, -2.5627412e-15,  2.5627412e-15],
       [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  2.5627412e-15,
        -2.5627412e-15, -2.5627412e-15,  2.5627412e-15],
       [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  2.5627412e-15,
        -2.5627412e-15, -2.5627412e-15,  2.5627412e-15],
       [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  2.5627412e-15,
        -2.5627412e-15, -2.5627412e-15,  2.5627412e-15],
       [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  2.5627412e-15,
        -2.5627412e-15, -

## Logarithm

In [5]:
def statistcs_Reps_Pertree_Logarithm(clf,n=100,p=4096,ratio=0.5,metric = 'mi',reps = 1):
    clf.reset()
    #  ## pauc is good 
    # coeffs = np.array([np.exp (-0.072 * (i + 10)) if i < 10 else 0 for i in range(p)])
    # coeffs = np.array([2 if i < 10 else 0 for i in range(p)])
    coeffs = np.array([10/(5+i) if i < 10 else 0 for i in range(p)])
    # coeffs = np.array([10/(i+1) if i < 10 else 0 for i in range(p)])
    # coeffs =np.array([5/(i+10) if i < 10 else 0 for i in range(p)])
  
    # coeffs = np.array([1/(i+5) if i < 10 else 0 for i in range(p)])
    coeffs_noise = np.array([1 if i >=10 else 0 for i in range(p)])
    # print(coeffs)

    x_1 = np.random.normal(size=(n, p))
    noise = np.random.normal(size=(n, p))

    x_2 = np.log((x_1 * coeffs+1)**2) + noise*coeffs_noise
    x = np.nan_to_num(np.float32(np.vstack((x_1,x_2))))
    y = np.array([0]*n+[1]*n).reshape(-1,1)

    if metric == 'auc':
        stats,pos,samples = clf.statistic(x, y, metric=metric, return_posteriors=True,max_fpr = 0.1)
    else:
        stats,pos,samples = clf.statistic(x, y, metric=metric,return_posteriors=True)
    clf.reset()
    POS = pos[:,:,0].reshape((n_estimators,2*n))

    # np.savetxt("/Users/baiyuxin/Desktop/JHU/NDD/Cancer/vs_samplesize/Posterior_log_samplesize_newcoef_p4096_{}_{}_{}.csv".format(metric,n, reps), POS, delimiter=",")
    # np.savetxt("/Users/baiyuxin/Desktop/JHU/NDD/Cancer/vs_samplesize/Samples_log_samplesize_newcoef_p4096_{}_{}_{}.csv".format(metric,n, reps), samples, delimiter=",")
    #POSs.append(POS)
    clf.reset()
    return stats

In [5]:
statistcs_Reps_Pertree_Logarithm(clf_Pertree,512,p=4096,ratio=0.5,metric = 'mi',reps = 1)

0.3426737460927717

In [6]:
REPs = 10
Stats_Paucs_Samplesize_Log = np.zeros((REPs,len(squared_values)))

for i in range(0,REPs):
    stats_paucs_samplesize_log =[]
    for n in squared_values:
        stat = statistcs_Reps_Pertree_Logarithm(clf_Pertree,n,p=4096,ratio=0.5,metric = 'mi',reps = i)
        # print(stat)
        stats_paucs_samplesize_log.append(stat)
    print(stats_paucs_samplesize_log)
    # np.savetxt("/Users/baiyuxin/Desktop/JHU/NDD/Cancer/vs_samplesize/log_pAUC_samplesize_newcoef_p4096_{}.csv".format(i), stats_paucs_samplesize_log, delimiter=",")
    Stats_Paucs_Samplesize_Log[i,:] = stats_paucs_samplesize_log

[0.0006417574358960065, 0.0005758080595051895, 0.0009033871668256976, 0.005722774218234639, 0.015144782519350808, 0.08587122251665058, 0.14193918948306883]
[0.0006873505745171871, 0.0006670048888640645, 0.0005891520296368302, 0.0021125739822639433, 0.03291759088580892, 0.088764850314624, 0.15552834392016146]
[0.0007581304850580173, 0.0006928378676906988, 0.0011255107335544912, 0.001608656286218202, 0.043122290817065756, 0.1116147538713036, 0.1672887680008811]


KeyboardInterrupt: 

In [6]:
from scipy.stats import entropy
Stats_Paucs_Log = np.zeros((REPs,len(squared_values)))
for i in range(REPs):
    pauc = np.array(np.genfromtxt('/Users/baiyuxin/Desktop/JHU/NDD/Cancer/vs_samplesize/log_pAUC_samplesize_newcoef_p4096_{}.csv'.format(i),delimiter=','))
    Stats_Paucs_Log[i,:] = pauc
print(Stats_Paucs_Log)


Stats_MI_Log = np.zeros((REPs,len(squared_values)))
Stats_Conen_Log = np.zeros((REPs,len(squared_values)))
for i in range(REPs):
    for samp in range(len(squared_values)):
        pos = np.genfromtxt('/Users/baiyuxin/Desktop/JHU/NDD/Cancer/vs_samplesize/Posterior_log_samplesize_newcoef_p4096_auc_{}_{}.csv'.format(squared_values[samp],i),delimiter=',')
        posterior_forest_0 = np.nanmean(pos, axis=0)
        posterior_forest_1 = np.ones(posterior_forest_0.shape)-posterior_forest_0
        posterior_forest = np.hstack((posterior_forest_0.reshape(-1,1),posterior_forest_1.reshape(-1,1)))

        stats_conen = np.mean(entropy(posterior_forest, base=np.exp(1), axis=1))

        H_Y = entropy([50,50], base=np.exp(1))
        stats_mi = H_Y - stats_conen

        Stats_Conen_Log[i,samp] = stats_conen
        Stats_MI_Log[i,samp] = stats_mi

[[1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1.]]


In [7]:
np.savetxt("/Users/baiyuxin/Desktop/JHU/NDD/Cancer/vs_samplesize/Stats_Paucs_SampleSize_Log_p4096_MIGHT.csv", Stats_Paucs_Log, delimiter=",")
np.savetxt("/Users/baiyuxin/Desktop/JHU/NDD/Cancer/vs_samplesize/Stats_Conen_SampleSize_Log_p4096_MIGHT.csv", Stats_Conen_Log, delimiter=",")
np.savetxt("/Users/baiyuxin/Desktop/JHU/NDD/Cancer/vs_samplesize/Stats_MI_SampleSize_Log_p4096_MIGHT.csv", Stats_MI_Log, delimiter=",")
Stats_MI_Log

array([[0.10230294, 0.32679467, 0.37428895, 0.49259378, 0.56720319,
        0.58739208, 0.62768722],
       [0.04670978, 0.32425425, 0.45807796, 0.50113577, 0.57317046,
        0.61084314, 0.62284952],
       [0.05769651, 0.33022246, 0.37327539, 0.50600449, 0.55010396,
        0.58936932, 0.62420175],
       [0.0788431 , 0.29028286, 0.40307391, 0.50753799, 0.56093851,
        0.59915906, 0.62866621],
       [0.06274998, 0.29480464, 0.37776669, 0.5137147 , 0.57267364,
        0.59525009, 0.62406214],
       [0.09092627, 0.31357664, 0.392891  , 0.51037266, 0.55618536,
        0.60344465, 0.6212433 ],
       [0.04372821, 0.34899949, 0.44604664, 0.48675269, 0.55456954,
        0.60838485, 0.63434864],
       [0.09758118, 0.37379904, 0.38965238, 0.49387394, 0.54718465,
        0.60305434, 0.61958493],
       [0.08575585, 0.3097991 , 0.35098228, 0.48810305, 0.54709057,
        0.59395181, 0.63145524],
       [0.09167018, 0.35328603, 0.4237754 , 0.49707596, 0.55297603,
        0.59916622, 0.6

In [8]:
### add KNN
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import roc_auc_score


p = 4096
REPs = 50
ratio = 0.5
sample_sizes = list(range(4,10,1))
# Square each element in the sequence
squared_values = [2 ** x for x in sample_sizes]

Stats_MI_Samplesize_Mu2_KNN = np.zeros((REPs,len(squared_values)))
Stats_Conen_Samplesize_Mu2_KNN = np.zeros((REPs,len(squared_values)))
Stats_Pauc_Samplesize_Mu2_KNN = np.zeros((REPs,len(squared_values)))


for i in range(0,REPs):
    print(i)
    stats_paucs_samplesize_mu0 =[]
    stats_mi_samplesize_mu0 =[]
    stats_conen_samplesize_mu0 =[]
    for n in squared_values:
        neigh = KNeighborsClassifier(n_neighbors=int(np.sqrt(n)))
        coeffs = np.array([np.exp(-0.072 * (i + 10)) if i < 10 else 0 for i in range(p)])
        coeffs_noise = np.array([1 if i >=10 else 0 for i in range(p)])
        # print(coeffs)

        x_1 = np.random.normal(size=(n, p))
        noise = np.random.normal(size=(n, p))

        x_2 = np.log((x_1 * coeffs+1)**2) + noise*coeffs_noise
        x = np.vstack((x_1,x_2))
        y = np.array([0]*n+[1]*n).reshape(-1,1).ravel()
        
        neigh.fit(x, y)
        posterior = neigh.predict_proba(x)

        stats_conen = np.mean(entropy(posterior, base=np.exp(1), axis=1))
        # print(stats_conen)

        # _, counts = np.unique(y_true_final, return_counts=True)
        H_Y = entropy([50,50], base=np.exp(1))
        # print(H_Y)
        stats_mi = H_Y - stats_conen

        pauc = roc_auc_score(
                y, posterior[:,1], max_fpr=0.1
            )
        print(stats_conen,stats_mi,pauc)
        
        stats_paucs_samplesize_mu0.append(pauc)
        stats_mi_samplesize_mu0.append(stats_mi)
        stats_conen_samplesize_mu0.append(stats_conen)

    Stats_Pauc_Samplesize_Mu2_KNN[i,:] = stats_paucs_samplesize_mu0
    Stats_MI_Samplesize_Mu2_KNN[i,:] = stats_mi_samplesize_mu0
    Stats_Conen_Samplesize_Mu2_KNN[i,:] = stats_conen_samplesize_mu0
    

0
0.5194369156268853 0.17371026493305997 0.5582853618421053
0.5953426444012728 0.09780453615867246 0.6907894736842105
0.6393425937804809 0.053804586779464425 0.6201878597861842
0.6413385451383407 0.05180863542160463 0.5765573601973684
0.6592917266252745 0.03385545393467082 0.5308667050594348
0.6690563959062266 0.024090784653718722 0.5348444737886128
1
0.49368819011122644 0.19945899044871884 0.756578947368421
0.5559764346424299 0.13717074591751544 0.6644736842105263
0.6359043173234599 0.05724286323648542 0.5263855439379699
0.6206281548068009 0.0725190257531444 0.5443846050061678
0.6677814844703509 0.02536569608959438 0.5565245778937089
0.6716101706249546 0.021537009934990636 0.5331179970189145
2
0.5762437115580591 0.1169034690018862 0.5752467105263158
0.6004644128398215 0.09268276772012374 0.6038240131578947
0.6210899586401619 0.07205722191978337 0.547620271381579
0.6448290434523316 0.04831813710761368 0.6004237124794408
0.6640107791030007 0.029136401456944627 0.5201659918296174
0.67040

In [9]:
np.savetxt("/Users/baiyuxin/Desktop/JHU/NDD/Cancer/vs_samplesize/Stats_Paucs_SampleSize_Log_KNN.csv", Stats_Pauc_Samplesize_Mu2_KNN, delimiter=",")
np.savetxt("/Users/baiyuxin/Desktop/JHU/NDD/Cancer/vs_samplesize/Stats_Conen_SampleSize_Log_KNN.csv", Stats_Conen_Samplesize_Mu2_KNN, delimiter=",")
np.savetxt("/Users/baiyuxin/Desktop/JHU/NDD/Cancer/vs_samplesize/Stats_MI_SampleSize_Log_KNN.csv", Stats_MI_Samplesize_Mu2_KNN, delimiter=",")
Stats_MI_Samplesize_Mu2_KNN

array([[0.17371026, 0.09780454, 0.05380459, 0.05180864, 0.03385545,
        0.02409078],
       [0.19945899, 0.13717075, 0.05724286, 0.07251903, 0.0253657 ,
        0.02153701],
       [0.11690347, 0.09268277, 0.07205722, 0.04831814, 0.0291364 ,
        0.02274074],
       [0.23051706, 0.16089938, 0.07704594, 0.05639916, 0.02415312,
        0.02225524],
       [0.21703196, 0.11613913, 0.08366413, 0.04669786, 0.03080094,
        0.02218364],
       [0.17371026, 0.13717075, 0.07182503, 0.05746244, 0.03333985,
        0.01983269],
       [0.11690347, 0.13232125, 0.08337008, 0.05416034, 0.03099762,
        0.0210416 ],
       [0.09646409, 0.06840959, 0.04702484, 0.04786344, 0.0275589 ,
        0.02144188],
       [0.13569791, 0.06301555, 0.05844038, 0.04198555, 0.0310289 ,
        0.02144326],
       [0.16431304, 0.14256478, 0.06002999, 0.04069522, 0.03004982,
        0.02122978],
       [0.10463984, 0.0924105 , 0.0581188 , 0.04510075, 0.03190877,
        0.02340777],
       [0.07889112, 0

In [10]:
##Add Logistics Regression for Log
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
REPs = 20
p = 4096

Stats_MI_LG = np.zeros((REPs,len(squared_values)))
Stats_Conen_LG = np.zeros((REPs,len(squared_values)))
Stats_Pauc_LG = np.zeros((REPs,len(squared_values)))


for i in range(0,REPs):
    print(i)
    stats_paucs_samplesize_mu0 =[]
    stats_mi_samplesize_mu0 =[]
    stats_conen_samplesize_mu0 =[]
    for n in squared_values:
        coeffs = np.array([np.exp(-0.072 * (i + 10)) if i <10 else 0 for i in range(p)])
        coeffs_noise = np.array([1 if i >= 10 else 0 for i in range(p)])

        x_1 = np.random.normal(size=(n, p))
        noise = np.random.normal(size=(n, p))
        x_2 = np.log((x_1 * coeffs+1)**2) + noise*coeffs_noise

        x = np.vstack((x_1,x_2))
        y = np.array([0]*n+[1]*n).reshape(-1,1).ravel()
        
        clf = LogisticRegression().fit(x, y)
        # print(clf.score(x, y))
        posterior = clf.predict_proba(x)

        stats_conen = np.mean(entropy(posterior, base=np.exp(1), axis=1))
        # print(stats_conen)

        # _, counts = np.unique(y_true_final, return_counts=True)
        H_Y = entropy([50,50], base=np.exp(1))
        # print(H_Y)
        stats_mi = H_Y - stats_conen

        pauc = roc_auc_score(
                y, posterior[:,1], max_fpr=0.1
            )
        print(stats_conen,stats_mi,pauc)
        
        stats_paucs_samplesize_mu0.append(pauc)
        stats_mi_samplesize_mu0.append(stats_mi)
        stats_conen_samplesize_mu0.append(stats_conen)

    Stats_Pauc_LG[i,:] = stats_paucs_samplesize_mu0
    Stats_MI_LG[i,:] = stats_mi_samplesize_mu0
    Stats_Conen_LG[i,:] = stats_conen_samplesize_mu0
    

0
0.011822414039236142 0.6813247665207092 1.0
0.01167492796739656 0.6814722525925487 1.0
0.012127603016537834 0.6810195775434075 1.0
0.012255862677933233 0.6808913178820121 1.0
0.012729918069262101 0.6804172624906832 1.0
0.013847569940965476 0.6792996106189798 1.0
1
0.011742331411086854 0.6814048491488585 1.0
0.01209557143373212 0.6810516091262132 1.0
0.011782883454397288 0.681364297105548 1.0
0.011838535286068723 0.6813086452738766 1.0
0.012663977650400218 0.6804832029095451 1.0
0.013786607593591807 0.6793605729663534 1.0
2
0.011965695350439953 0.6811814852095053 1.0
0.011809916229832525 0.6813372643301128 1.0
0.012402023060387431 0.6807451574995579 1.0
0.012302903266135534 0.6808442772938097 1.0
0.012427363563157668 0.6807198169967876 1.0
0.013337825114714088 0.6798093554452312 1.0
3
0.011909548667328188 0.6812376318926171 1.0
0.011765676867561838 0.6813815036923835 1.0
0.011850465401524254 0.681296715158421 1.0
0.012099370363677166 0.6810478101962681 1.0
0.012470389367916486 0.68067

In [11]:
np.savetxt("/Users/baiyuxin/Desktop/JHU/NDD/Cancer/vs_samplesize/Stats_Paucs_SampleSize_Log_LG.csv", Stats_Pauc_LG, delimiter=",")
np.savetxt("/Users/baiyuxin/Desktop/JHU/NDD/Cancer/vs_samplesize/Stats_Conen_SampleSize_Log_LG.csv", Stats_Conen_LG, delimiter=",")
np.savetxt("/Users/baiyuxin/Desktop/JHU/NDD/Cancer/vs_samplesize/Stats_MI_SampleSize_Log_LG.csv", Stats_MI_LG, delimiter=",")
Stats_MI_LG

array([[0.68132477, 0.68147225, 0.68101958, 0.68089132, 0.68041726,
        0.67929961],
       [0.68140485, 0.68105161, 0.6813643 , 0.68130865, 0.6804832 ,
        0.67936057],
       [0.68118149, 0.68133726, 0.68074516, 0.68084428, 0.68071982,
        0.67980936],
       [0.68123763, 0.6813815 , 0.68129672, 0.68104781, 0.68067679,
        0.67975865],
       [0.68146931, 0.68125077, 0.68144622, 0.6810511 , 0.68110778,
        0.67915299],
       [0.6813103 , 0.68124982, 0.68157762, 0.68129787, 0.680421  ,
        0.67925651],
       [0.68139359, 0.68159739, 0.68126421, 0.68130753, 0.68061351,
        0.67992556],
       [0.68130282, 0.68129007, 0.68147533, 0.68092568, 0.68015054,
        0.67949287],
       [0.68161398, 0.6813709 , 0.68114   , 0.68112005, 0.68065912,
        0.67908867],
       [0.68129968, 0.68114742, 0.6810679 , 0.6813288 , 0.68098989,
        0.67889717],
       [0.68172732, 0.68137265, 0.68135721, 0.68069604, 0.68072725,
        0.67980179],
       [0.6812719 , 0

In [5]:
### Add SVM to the Log
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import cross_val_score, StratifiedKFold, cross_val_predict

REPs = 20
p = 4096

Stats_MI_SVM = np.zeros((REPs,len(squared_values)))
Stats_Conen_SVM = np.zeros((REPs,len(squared_values)))
Stats_Pauc_SVM = np.zeros((REPs,len(squared_values)))


for i in range(0,REPs):
    print(i)
    stats_paucs_samplesize_mu0 =[]
    stats_mi_samplesize_mu0 =[]
    stats_conen_samplesize_mu0 =[]
    for n in squared_values:
        
        coeffs = np.array([np.exp(-0.072 * (i + 10)) if i <10 else 0 for i in range(p)])
        coeffs_noise = np.array([1 if i >= 10 else 0 for i in range(p)])

        x_1 = np.random.normal(size=(n, p))
        noise = np.random.normal(size=(n, p))
        x_2 = np.log((x_1 * coeffs+1)**2) + noise*coeffs_noise

        x = np.vstack((x_1,x_2))
        y = np.array([0]*n+[1]*n).reshape(-1,1).ravel()
        
        model = SVC(probability=True,kernel = 'rbf')  # Set probability=True to enable probability estimates
        model.fit(x, y)
        posterior = model.predict_proba(x)

        stats_conen = np.mean(entropy(posterior, base=np.exp(1), axis=1))
        # print(stats_conen)

        # _, counts = np.unique(y_true_final, return_counts=True)
        H_Y = entropy([50,50], base=np.exp(1))
        # print(H_Y)
        stats_mi = H_Y - stats_conen

        pauc = roc_auc_score(
                y, posterior[:,1], max_fpr=0.1
            )
        print(stats_conen,stats_mi,pauc)
        
        stats_paucs_samplesize_mu0.append(pauc)
        stats_mi_samplesize_mu0.append(stats_mi)
        stats_conen_samplesize_mu0.append(stats_conen)

    Stats_Pauc_SVM[i,:] = stats_paucs_samplesize_mu0
    Stats_MI_SVM[i,:] = stats_mi_samplesize_mu0
    Stats_Conen_SVM[i,:] = stats_conen_samplesize_mu0
    

0
0.2758277455985745 0.4173194349613708 0.47368421052631576
0.42421541955557357 0.2689317610043717 0.47368421052631576
0.638998953195976 0.05414822736396929 0.47368421052631576
0.5451960782671581 0.14795110229278718 0.47368421052631576
0.670379284617861 0.022767895942084282 0.47368421052631576
0.6695017787552828 0.023645401804662458 0.47368421052631576
1
0.3663742396128662 0.3267729409470791 0.47368421052631576
0.46489318509118593 0.22825399546875935 0.47368421052631576
0.5967259069797282 0.09642127358021713 0.47368421052631576
0.6699854209199096 0.02316175964003564 0.47368421052631576
0.692762680016171 0.0003845005437742577 0.47368421052631576
0.637334878178901 0.05581230238104429 1.0
2
0.21593409866725632 0.47721308189268896 0.47368421052631576
0.3374892265157714 0.3556579540441739 0.47368421052631576
0.4870805320385853 0.20606664852135997 0.47368421052631576
0.6564290848226729 0.03671809573727236 0.47368421052631576
0.6234033885544253 0.06974379200551994 0.47368421052631576
0.614669

In [6]:
np.savetxt("/Users/baiyuxin/Desktop/JHU/NDD/Cancer/vs_samplesize/Stats_Paucs_SampleSize_Log_SVM.csv", Stats_Pauc_SVM, delimiter=",")
np.savetxt("/Users/baiyuxin/Desktop/JHU/NDD/Cancer/vs_samplesize/Stats_Conen_SampleSize_Log_SVM.csv", Stats_Conen_SVM, delimiter=",")
np.savetxt("/Users/baiyuxin/Desktop/JHU/NDD/Cancer/vs_samplesize/Stats_MI_SampleSize_Log_SVM.csv", Stats_MI_SVM, delimiter=",")
Stats_MI_SVM

array([[4.17319435e-01, 2.68931761e-01, 5.41482274e-02, 1.47951102e-01,
        2.27678959e-02, 2.36454018e-02],
       [3.26772941e-01, 2.28253995e-01, 9.64212736e-02, 2.31617596e-02,
        3.84500544e-04, 5.58123024e-02],
       [4.77213082e-01, 3.55657954e-01, 2.06066649e-01, 3.67180957e-02,
        6.97437920e-02, 7.84780699e-02],
       [2.90505786e-01, 1.45082989e-01, 2.29344454e-01, 1.82706923e-01,
        2.17517386e-01, 7.18863077e-03],
       [3.99454583e-01, 1.62171469e-01, 2.85623307e-01, 1.29261479e-01,
        1.14540886e-01, 1.10338556e-01],
       [4.19002236e-01, 2.69484774e-01, 2.92056033e-01, 4.16887914e-02,
        4.18604840e-02, 1.37579114e-03],
       [3.24496206e-01, 4.45321857e-01, 1.54149287e-01, 2.16955809e-03,
        1.58246224e-01, 6.00874063e-02],
       [4.52097376e-01, 2.77884484e-01, 1.14257929e-01, 1.33494199e-02,
        2.83970527e-02, 3.57656464e-02],
       [4.34450900e-01, 3.27879305e-01, 1.07123486e-01, 2.66491694e-02,
        5.77232736e-02, 

In [41]:
### KSG for Mutual Information
from sktree.experimental import mutual_info_ksg
from sklearn.feature_selection import mutual_info_classif
from sklearn.feature_selection import mutual_info_regression
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import cross_val_score, StratifiedKFold, cross_val_predict
from sktree.ensemble import UnsupervisedObliqueRandomForest
from sktree.tree import compute_forest_similarity_matrix


REPs = 10
p = 4096

Stats_MI_KSG = np.zeros((REPs,len(squared_values)))

for i in range(0,REPs):
    print(i)
    stats_mi_samplesize_mu0 =[]
    for n in squared_values:
        # coeffs = np.array([np.exp(-0.072 * (i + 10)) if i <10 else 0 for i in range(p)])
        coeffs = np.array([100/(i+1) if i < 10 else 0 for i in range(p)])
        coeffs_noise = np.array([1 if i >= 10 else 0 for i in range(p)])

        x_1 = np.random.normal(size=(n, p))
        noise = np.random.normal(size=(n, p))
        x_2 = np.log((x_1 * coeffs+1)**2) + noise*coeffs_noise

        x = np.vstack((x_1,x_2))
        y = np.array([0]*n+[1]*n).reshape(-1,1).ravel()
        # stats_mi = mutual_info_classif(x,y,n_neighbors= int(np.sqrt(n)))
        stats_mi = mi(x,y,k = int(np.sqrt(n)))
        stats_mi_samplesize_mu0.append(np.mean(stats_mi))
    print(stats_mi_samplesize_mu0)
    Stats_MI_KSG[i,:] = stats_mi_samplesize_mu0

0
[0.0, 0.0, 0.0, 2.562741203051934e-15, -2.562741203051934e-15, -2.562741203051934e-15, 2.562741203051934e-15]
1
[0.0, 0.0, 0.0, 2.562741203051934e-15, -2.562741203051934e-15, -2.562741203051934e-15, 2.562741203051934e-15]
2
[0.0, 0.0, 0.0, 2.562741203051934e-15, -2.562741203051934e-15, -2.562741203051934e-15, 2.562741203051934e-15]
3
[0.0, 0.0, 0.0, 2.562741203051934e-15, -2.562741203051934e-15, -2.562741203051934e-15, -2.2013779307876766e-05]
4
[0.0, 0.0, 0.0, 2.562741203051934e-15, -2.562741203051934e-15, -2.562741203051934e-15, -2.2013779307876766e-05]
5
[0.0, 0.0, 0.0, 2.562741203051934e-15, -2.562741203051934e-15, -2.562741203051934e-15, 2.562741203051934e-15]
6
[0.0, 0.0, 0.0, 2.562741203051934e-15, -2.562741203051934e-15, -2.562741203051934e-15, 2.562741203051934e-15]
7
[0.0, 0.0, 0.0, 2.562741203051934e-15, -2.562741203051934e-15, -2.562741203051934e-15, 2.562741203051934e-15]
8
[0.0, 0.0, 0.0, 2.562741203051934e-15, -2.562741203051934e-15, -6.404008526859691e-05, 2.562741203

In [42]:
np.savetxt("/Users/baiyuxin/Desktop/JHU/NDD/Cancer/vs_samplesize/Stats_MI_SampleSize_Log_p4096_KSG.csv", Stats_MI_KSG, delimiter=",")
Stats_MI_KSG

array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         2.56274120e-15, -2.56274120e-15, -2.56274120e-15,
         2.56274120e-15],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         2.56274120e-15, -2.56274120e-15, -2.56274120e-15,
         2.56274120e-15],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         2.56274120e-15, -2.56274120e-15, -2.56274120e-15,
         2.56274120e-15],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         2.56274120e-15, -2.56274120e-15, -2.56274120e-15,
        -2.20137793e-05],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         2.56274120e-15, -2.56274120e-15, -2.56274120e-15,
        -2.20137793e-05],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         2.56274120e-15, -2.56274120e-15, -2.56274120e-15,
         2.56274120e-15],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         2.56274120e-15, -2.56274120e-15, -2.56274120e-15,
         2.5627412

## Multi-independent

In [3]:
### KSG for Mutual Information
from sktree.experimental import mutual_info_ksg
from sklearn.feature_selection import mutual_info_classif
from sklearn.feature_selection import mutual_info_regression
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import cross_val_score, StratifiedKFold, cross_val_predict

REPs = 10
p = 4096

Stats_MI_KSG = np.zeros((REPs,len(squared_values)))

for i in range(0,REPs):
    print(i)
    stats_mi_samplesize_mu0 =[]
    for n in squared_values:
        prob=0.5
        sep1=3 
        sep2=2
        rng = np.random.default_rng()
        sig = np.identity(p)
        u = rng.multivariate_normal(np.zeros(p), sig, size=n, method='cholesky')
        v = rng.multivariate_normal(np.zeros(p), sig, size=n, method='cholesky')
        u_2 = rng.binomial(1, prob, size=(n, p))
        v_2 = rng.binomial(1, prob, size=(n, p))

        x_1 = u / sep1 + sep2 * u_2 - 1
        x_2 = v / sep1 + sep2 * v_2 - 1
        x = np.vstack((u,v))
        y = np.array([0]*n+[1]*n).reshape(-1,1).ravel()

        # stats_mi = mutual_info_classif(x,y,n_neighbors= int(np.sqrt(n)))
        stats_mi = _compute_mi_cc(x_1,x_2,n_neighbors= int(np.sqrt(n)))
        print(stats_mi) 
        stats_mi_samplesize_mu0.append(np.mean(stats_mi))
    Stats_MI_KSG[i,:] = stats_mi_samplesize_mu0



    

0


NameError: name '_compute_mi_cc' is not defined

In [10]:
np.savetxt("/Users/baiyuxin/Desktop/JHU/NDD/Cancer/vs_samplesize/Stats_MI_SampleSize_MultiInd_KSG.csv", Stats_MI_KSG, delimiter=",")
Stats_MI_KSG

array([[0.02841474, 0.01849996, 0.01041223, 0.006518  , 0.00383969,
        0.00254855],
       [0.02694265, 0.01810757, 0.01028722, 0.00639433, 0.00400384,
        0.00241228],
       [0.03008336, 0.01849742, 0.01005565, 0.00674076, 0.00387243,
        0.00240096],
       [0.02829023, 0.01965002, 0.01088954, 0.00663375, 0.00388637,
        0.002437  ],
       [0.02903164, 0.01883031, 0.01047555, 0.00677549, 0.00384575,
        0.00246472],
       [0.02812307, 0.01746524, 0.01055003, 0.00654299, 0.00403053,
        0.00235369],
       [0.02727802, 0.01916718, 0.0106587 , 0.00643484, 0.00395242,
        0.0023987 ],
       [0.02940965, 0.0191306 , 0.01070659, 0.00656919, 0.0040815 ,
        0.00240505],
       [0.02830998, 0.01870528, 0.01107726, 0.00680717, 0.00395672,
        0.00244205],
       [0.02837215, 0.0186479 , 0.01093663, 0.00672777, 0.003772  ,
        0.00241096]])

In [14]:
def statistcs_Reps_Pertree_Gaussian(clf,n=100,p=4096,ratio=0.5,metric = 'mi',reps = 1):
    prob=0.5
    sep1=3 
    sep2=2
    clf.reset()
    rng = np.random.default_rng()
    sig = np.identity(p)
    u = rng.multivariate_normal(np.zeros(p), sig, size=n, method='cholesky')
    v = rng.multivariate_normal(np.zeros(p), sig, size=n, method='cholesky')
    u_2 = rng.binomial(1, prob, size=(n, p))
    v_2 = rng.binomial(1, prob, size=(n, p))

    x_1 = u / sep1 + sep2 * u_2 - 1
    x_2 = v / sep1 + sep2 * v_2 - 1
    x = np.vstack((x_1,x_2))
    y = np.array([0]*n+[1]*n).reshape(-1,1)

    if metric == 'auc':
        stats,pos,samples = clf.statistic(x, y, metric=metric, return_posteriors=True,max_fpr = 0.1)
    else:
        stats,pos,samples = clf.statistic(x, y, metric=metric,return_posteriors=True)
    clf.reset()
    POS = pos[:,:,0].reshape((n_estimators,2*n))

    np.savetxt("/Users/baiyuxin/Desktop/JHU/NDD/Cancer/vs_samplesize/Posterior_MultiInd_samplesize_{}_{}_{}.csv".format(metric,n, reps), POS, delimiter=",")
    np.savetxt("/Users/baiyuxin/Desktop/JHU/NDD/Cancer/vs_samplesize/Samples_MultiInd_samplesize_{}_{}_{}.csv".format(metric,n, reps), samples, delimiter=",")

    #POSs.append(POS)
    clf.reset()
    return stats

In [15]:
REPs = 10
Stats_Paucs_Samplesize_Gaussian = np.zeros((REPs,len(squared_values)))

for i in range(0,REPs):
    print(i)
    stats_paucs_samplesize_gaussian =[]
    for n_i in squared_values:
        print(n_i)
        stat = statistcs_Reps_Pertree_Gaussian(clf_Pertree,n_i,p = 4096,ratio=0.5,metric = 'auc',reps = i)
        stats_paucs_samplesize_gaussian.append(stat) 
    print(stats_paucs_samplesize_gaussian)
    np.savetxt("/Users/baiyuxin/Desktop/JHU/NDD/Cancer/vs_samplesize/MultiInd_pAUC_samplesize_{}.csv".format(i), stats_paucs_samplesize_gaussian, delimiter=",")
    Stats_Paucs_Samplesize_Gaussian[i,:] = stats_paucs_samplesize_gaussian

0
16
32
64
128
256
512
1024
[0.47368421052631576, 0.4849917763157895, 0.5029810855263158, 0.5143528988486842, 0.5138710423519737, 0.5049968518708882, 0.5101818285490337]
1
16
32
64
128
256
512
1024
[0.48601973684210525, 0.4973273026315789, 0.5027240953947368, 0.4905813116776316, 0.4897621556332237, 0.4959981817948191, 0.503098538047389]
2
16
32
64
128
256
512
1024
[0.48601973684210525, 0.5446134868421053, 0.5055509868421053, 0.4991904810855263, 0.4984676963404605, 0.4977288497121711, 0.5064394097579153]
3
16
32
64
128
256
512
1024
[0.53125, 0.4973273026315789, 0.5243112664473685, 0.503430818256579, 0.4928942228618421, 0.49738351922286184, 0.5015977558336759]
4
16
32
64
128
256
512
1024
[0.5189144736842105, 0.506578947368421, 0.5351048519736842, 0.5305432771381579, 0.49551230982730265, 0.500680220754523, 0.49968137239155014]
5
16
32
64
128
256
512
1024
[0.48601973684210525, 0.5127467105263158, 0.5361328125, 0.5027240953947368, 0.48004471628289475, 0.4944723028885691, 0.496102584035773]


In [16]:
Stats_Paucs_Log = np.zeros((REPs,len(squared_values)))
for i in range(REPs):
    pauc = np.array(np.genfromtxt('/Users/baiyuxin/Desktop/JHU/NDD/Cancer/vs_samplesize/MultiInd_pAUC_samplesize_{}.csv'.format(i),delimiter=','))
    Stats_Paucs_Log[i,:] = pauc
print(Stats_Paucs_Log)


Stats_MI_Log = np.zeros((REPs,len(squared_values)))
Stats_Conen_Log = np.zeros((REPs,len(squared_values)))
for i in range(REPs):
    for samp in range(len(squared_values)):
        pos = np.genfromtxt('/Users/baiyuxin/Desktop/JHU/NDD/Cancer/vs_samplesize/Posterior_MultiInd_samplesize_auc_{}_{}.csv'.format(squared_values[samp],i),delimiter=',')
        posterior_forest_0 = np.nanmean(pos, axis=0)
        posterior_forest_1 = np.ones(posterior_forest_0.shape)-posterior_forest_0
        posterior_forest = np.hstack((posterior_forest_0.reshape(-1,1),posterior_forest_1.reshape(-1,1)))

        stats_conen = np.mean(entropy(posterior_forest, base=np.exp(1), axis=1))

        H_Y = entropy([50,50], base=np.exp(1))
        stats_mi = H_Y - stats_conen

        Stats_Conen_Log[i,samp] = stats_conen
        Stats_MI_Log[i,samp] = stats_mi

[[0.47368421 0.48499178 0.50298109 0.5143529  0.51387104 0.50499685
  0.51018183]
 [0.48601974 0.4973273  0.5027241  0.49058131 0.48976216 0.49599818
  0.50309854]
 [0.48601974 0.54461349 0.50555099 0.49919048 0.4984677  0.49772885
  0.50643941]
 [0.53125    0.4973273  0.52431127 0.50343082 0.49289422 0.49738352
  0.50159776]
 [0.51891447 0.50657895 0.53510485 0.53054328 0.49551231 0.50068022
  0.49968137]
 [0.48601974 0.51274671 0.53613281 0.5027241  0.48004472 0.4944723
  0.49610258]
 [0.47368421 0.50657895 0.49552837 0.52315481 0.49890137 0.50261969
  0.50463345]
 [0.47368421 0.50657895 0.51223273 0.48075144 0.50376812 0.49652019
  0.50113096]
 [0.47368421 0.47985197 0.4998972  0.50741417 0.50195312 0.49535571
  0.50291483]
 [0.51069079 0.52302632 0.52508224 0.49848376 0.5131322  0.50355128
  0.5003881 ]]


In [19]:
np.savetxt("/Users/baiyuxin/Desktop/JHU/NDD/Cancer/vs_samplesize/Stats_Paucs_SampleSize_MultiInd_p2048_MIGHT.csv", Stats_Paucs_Log, delimiter=",")
np.savetxt("/Users/baiyuxin/Desktop/JHU/NDD/Cancer/vs_samplesize/Stats_Conen_SampleSize_MultiInd_p2048_MIGHT.csv", Stats_Conen_Log, delimiter=",")
np.savetxt("/Users/baiyuxin/Desktop/JHU/NDD/Cancer/vs_samplesize/Stats_MI_SampleSize_MultiInd_p2048_MIGHT.csv", Stats_MI_Log, delimiter=",")
Stats_MI_Log

array([[0.00066042, 0.00050529, 0.00056279, 0.00057733, 0.00052003,
        0.00047161, 0.00051766],
       [0.00092974, 0.00038519, 0.00049662, 0.00062691, 0.00044143,
        0.00051002, 0.00050416],
       [0.00075407, 0.0008858 , 0.00069737, 0.00042036, 0.00047743,
        0.00049997, 0.00047515],
       [0.00055685, 0.00040719, 0.0005831 , 0.00054645, 0.00052225,
        0.00047837, 0.00050367],
       [0.00098346, 0.00098864, 0.00058452, 0.00043396, 0.00053608,
        0.00047528, 0.00048785],
       [0.00047661, 0.00066149, 0.00068564, 0.00058615, 0.00053077,
        0.00049906, 0.00046237],
       [0.00082728, 0.00070234, 0.00054631, 0.00051523, 0.00048552,
        0.00049857, 0.00047433],
       [0.00064834, 0.00048155, 0.00048459, 0.00042449, 0.00051923,
        0.00049094, 0.00049079],
       [0.00033855, 0.0006124 , 0.00061384, 0.00048725, 0.00052625,
        0.00043957, 0.00046898],
       [0.00071565, 0.00043447, 0.0005814 , 0.00055217, 0.00050748,
        0.00046996, 0.0

### Gaussian Mu=0

In [7]:
### Add Logistic Regression
##Add Logistics Regression for Logarithm
#### change n
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
REPs = 50
p = 4096
Stats_MI_DIM_Mu2_LG = np.zeros((REPs,len(squared_values)))
Stats_Conen_DIM_Mu2_LG = np.zeros((REPs,len(squared_values)))
Stats_Pauc_DIM_Mu2_LG = np.zeros((REPs,len(squared_values)))


for i in range(0,REPs):
    print(i)
    stats_paucs_samplesize_mu0 =[]
    stats_mi_samplesize_mu0 =[]
    stats_conen_samplesize_mu0 =[]
    for p in squared_values:
        x_1 = np.random.normal(size=(n, p))
        x_2 = np.random.normal(size=(n, p))
        x = np.vstack((x_1,x_2))
        y = np.array([0]*n+[1]*n).reshape(-1,1).ravel()
        
        clf = LogisticRegression().fit(x, y)
        print(clf.score(x, y))
        posterior = clf.predict_proba(x)

        stats_conen = np.mean(entropy(posterior, base=np.exp(1), axis=1))
        # print(stats_conen)

        # _, counts = np.unique(y_true_final, return_counts=True)
        H_Y = entropy([50,50], base=np.exp(1))
        # print(H_Y)
        stats_mi = H_Y - stats_conen

        pauc = roc_auc_score(
                y, posterior[:,1], max_fpr=0.1
            )
        print(stats_conen,stats_mi,pauc)
        
        stats_paucs_samplesize_mu0.append(pauc)
        stats_mi_samplesize_mu0.append(stats_mi)
        stats_conen_samplesize_mu0.append(stats_conen)

    Stats_Pauc_DIM_Mu2_LG[i,:] = stats_paucs_samplesize_mu0
    Stats_MI_DIM_Mu2_LG[i,:] = stats_mi_samplesize_mu0
    Stats_Conen_DIM_Mu2_LG[i,:] = stats_conen_samplesize_mu0
    

0
0.5595703125
0.6867662038669937 0.006380976692951568 0.5191232781661185
0.568359375
0.6770649766621955 0.016082203897749836 0.5250661749588815
0.59765625
0.66096089591882 0.03218628464112527 0.5352494089226973
0.630859375
0.6252706323507408 0.06787654820920452 0.5775604248046875
0.7197265625
0.5470699746083607 0.14607720595158458 0.6410851729543585
0.86328125
0.34744270544761346 0.3457044751123318 0.8262786865234375
1
0.5615234375
0.6814353874954082 0.011711793064537068 0.5286680522717928
0.5703125
0.6720145022991912 0.021132678260754045 0.5319567228618421
0.5966796875
0.6632740088877256 0.0298731716722197 0.5331533331620066
0.671875
0.6082197747690657 0.08492740579087954 0.5853504381681743
0.740234375
0.5491910297205838 0.14395615083936153 0.6043155067845395
0.884765625
0.34394247236592584 0.34920470819401944 0.8712840833162007
2
0.5546875
0.6853472263540386 0.007799954205906734 0.5191594174033717
0.5556640625
0.6785104332689688 0.014636747290976526 0.5264274195620888
0.6123046875
0

In [8]:
np.savetxt("/Users/baiyuxin/Desktop/JHU/NDD/Cancer/vs_samplesize/Stats_Paucs_SampleSize_Gaussian_LG.csv", Stats_Pauc_DIM_Mu2_LG, delimiter=",")
np.savetxt("/Users/baiyuxin/Desktop/JHU/NDD/Cancer/vs_samplesize/Stats_Conen_SampleSize_Gaussian_LG.csv", Stats_Conen_DIM_Mu2_LG, delimiter=",")
np.savetxt("/Users/baiyuxin/Desktop/JHU/NDD/Cancer/vs_samplesize/Stats_MI_SampleSize_Gaussian_LG.csv", Stats_MI_DIM_Mu2_LG, delimiter=",")
Stats_MI_DIM_Mu2_LG

array([[0.00638098, 0.0160822 , 0.03218628, 0.06787655, 0.14607721,
        0.34570448],
       [0.01171179, 0.02113268, 0.02987317, 0.08492741, 0.14395615,
        0.34920471],
       [0.00779995, 0.01463675, 0.040369  , 0.06776295, 0.14971156,
        0.38416571],
       [0.00696639, 0.01898701, 0.02695616, 0.07204041, 0.14165844,
        0.36926617],
       [0.00568403, 0.01816287, 0.03554477, 0.06351085, 0.16597175,
        0.3891137 ],
       [0.00608599, 0.01020036, 0.02602548, 0.07927392, 0.13657084,
        0.37175484],
       [0.01139964, 0.01596469, 0.03768117, 0.06830094, 0.1457379 ,
        0.34348806],
       [0.01095995, 0.01829243, 0.03327795, 0.05067632, 0.14324029,
        0.3534419 ],
       [0.00934184, 0.01098262, 0.03526319, 0.06772145, 0.12003581,
        0.36328151],
       [0.00749632, 0.01660099, 0.0341332 , 0.0629804 , 0.14256961,
        0.38385877],
       [0.00795174, 0.01388957, 0.03871675, 0.08009723, 0.1613586 ,
        0.40984904],
       [0.00472923, 0

## Gaussian Mu=2

In [9]:
### Add Logistic Regression
##Add Logistics Regression for Logarithm
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
REPs = 50
p = 4096
Stats_MI_DIM_Mu2_LG = np.zeros((REPs,len(squared_values)))
Stats_Conen_DIM_Mu2_LG = np.zeros((REPs,len(squared_values)))
Stats_Pauc_DIM_Mu2_LG = np.zeros((REPs,len(squared_values)))


for i in range(0,REPs):
    print(i)
    stats_paucs_samplesize_mu0 =[]
    stats_mi_samplesize_mu0 =[]
    stats_conen_samplesize_mu0 =[]
    for p in squared_values:
        x_1 = np.random.normal(loc = np.array([2]*1+[0]*(p-1)),scale = np.array([1]*p),size=(n,p))
        x_2 = np.random.normal(loc = np.array([-2]*1+[0]*(p-1)),scale = np.array([1]*p),size=(n,p))
        x = np.vstack((x_1,x_2))
        y = np.array([0]*n+[1]*n).reshape(-1,1).ravel()
        
        clf = LogisticRegression().fit(x, y)
        print(clf.score(x, y))
        posterior = clf.predict_proba(x)

        stats_conen = np.mean(entropy(posterior, base=np.exp(1), axis=1))
        # print(stats_conen)

        # _, counts = np.unique(y_true_final, return_counts=True)
        H_Y = entropy([50,50], base=np.exp(1))
        # print(H_Y)
        stats_mi = H_Y - stats_conen

        pauc = roc_auc_score(
                y, posterior[:,1], max_fpr=0.1
            )
        print(stats_conen,stats_mi,pauc)
        
        stats_paucs_samplesize_mu0.append(pauc)
        stats_mi_samplesize_mu0.append(stats_mi)
        stats_conen_samplesize_mu0.append(stats_conen)

    Stats_Pauc_DIM_Mu2_LG[i,:] = stats_paucs_samplesize_mu0
    Stats_MI_DIM_Mu2_LG[i,:] = stats_mi_samplesize_mu0
    Stats_Conen_DIM_Mu2_LG[i,:] = stats_conen_samplesize_mu0
    

0
0.9853515625
0.07130114381060604 0.6218460367493392 0.9895959151418585
0.982421875
0.059792256290321005 0.6333549242696243 0.9958841424239309
0.9970703125
0.0527100469516189 0.6404371336083264 0.9993976793791118
1.0
0.034835453156809394 0.6583117274031359 1.0
1.0
0.030503384320914144 0.6626437962390311 1.0
1.0
0.021215280513758775 0.6719319000461865 1.0
1
0.9873046875
0.05582125381192558 0.6373259267480197 0.9965025249280428
0.9833984375
0.06776575063770948 0.6253814299222358 0.9919489810341282
0.9921875
0.05802497110781791 0.6351222094521274 0.9978316457648027
1.0
0.03964468910801949 0.6535024914519258 1.0
1.0
0.02855625930959657 0.6645909212503487 1.0
1.0
0.02203640623097334 0.6711107743289719 1.0
2
0.9716796875
0.08158480585419091 0.6115623747057544 0.9866646214535362
0.98046875
0.06681696719465616 0.6263302133652892 0.992571379009046
0.9853515625
0.07040046886189033 0.622746711698055 0.9938723915501644
1.0
0.035119633530034264 0.6580275470299111 1.0
1.0
0.03203996066285632 0.6611

In [10]:
np.savetxt("/Users/baiyuxin/Desktop/JHU/NDD/Cancer/vs_samplesize/Stats_Paucs_SampleSize_Gaussian_mu2_LG.csv", Stats_Pauc_DIM_Mu2_LG, delimiter=",")
np.savetxt("/Users/baiyuxin/Desktop/JHU/NDD/Cancer/vs_samplesize/Stats_Conen_SampleSize_Gaussian_mu2_LG.csv", Stats_Conen_DIM_Mu2_LG, delimiter=",")
np.savetxt("/Users/baiyuxin/Desktop/JHU/NDD/Cancer/vs_samplesize/Stats_MI_SampleSize_Gaussian_mu2_LG.csv", Stats_MI_DIM_Mu2_LG, delimiter=",")
Stats_MI_DIM_Mu2_LG

array([[0.62184604, 0.63335492, 0.64043713, 0.65831173, 0.6626438 ,
        0.6719319 ],
       [0.63732593, 0.62538143, 0.63512221, 0.65350249, 0.66459092,
        0.67111077],
       [0.61156237, 0.62633021, 0.62274671, 0.65802755, 0.66110722,
        0.67236353],
       [0.6332019 , 0.63658657, 0.63658365, 0.644461  , 0.66518268,
        0.67220651],
       [0.61862042, 0.64059184, 0.63934418, 0.64414368, 0.66214764,
        0.6716321 ],
       [0.63038094, 0.61906725, 0.62845529, 0.64934021, 0.66067592,
        0.672467  ],
       [0.6309    , 0.61968301, 0.64736558, 0.65617108, 0.66575679,
        0.67062516],
       [0.63743803, 0.62661809, 0.63867027, 0.64178687, 0.66393634,
        0.67166205],
       [0.62431371, 0.62433201, 0.64034551, 0.65907131, 0.66454025,
        0.67189394],
       [0.62796083, 0.62334752, 0.63796717, 0.65469836, 0.66258093,
        0.67123286],
       [0.63146329, 0.62671652, 0.62189771, 0.65124282, 0.66754578,
        0.67223626],
       [0.63267813, 0